In [1]:
import pandas as pd
from surprise import SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import accuracy
import math
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_pickle('C:\\Users\\ineso\\FEUP-3ano\\gulbenkian-ai\\data\\vegas-restaurants\\vegas_review_reduced.pickle')
df = df.sample(n=20000, random_state=1)

trainset, testset = train_test_split(df, test_size=0.25)

In [3]:
rest_df = pd.read_pickle('C:\\Users\\ineso\\FEUP-3ano\\gulbenkian-ai\\data\\vegas-restaurants\\restaurant_in_vegas.pickle')
rest_df = rest_df[['business_id', 'categories']]

In [14]:
cat_trainset = pd.DataFrame(columns=['user_id', 'category', 'stars'])
temp = trainset.merge(rest_df, how='inner', on='business_id')
i=0
for index, row in temp.iterrows():
    categories = row['categories'].split(", ")
    for c in categories:
        cat_trainset.loc[i] = [row['user_id'], c, row['stars']]
        i+=1

KeyError: 'stars'

In [ ]:
cat_train_data = Dataset.load_from_df(cat_trainset[['user_id', 'category', 'stars']], reader)
cat_train_set = cat_train_data.build_full_trainset()

In [4]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'business_id', 'stars']], reader)
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE', 'MAE'], cv=5, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')   

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Don

,test_rmse,test_mae,fit_time,test_time
Algorithm,,,,
KNNBaseline,1.344533,1.105023,81.542561,2.769784
SVDpp,1.344707,1.104956,15.117190,0.295407
SVD,1.344936,1.103301,7.946307,0.205485
BaselineOnly,1.345361,1.105783,0.216216,0.074292
KNNBasic,1.392942,1.162765,78.576137,2.507703
SlopeOne,1.434583,1.182790,3.514133,0.256829
KNNWithMeans,1.436324,1.183402,53.585435,1.183902
KNNWithZScore,1.437116,1.182926,32.206551,0.690425
NMF,1.438580,1.200136,22.539212,0.282161


In [9]:
reader = Reader(rating_scale=(1, 5))
train_data = Dataset.load_from_df(trainset[['user_id', 'business_id', 'stars']], reader)
train_set = train_data.build_full_trainset()

test_data = Dataset.load_from_df(testset[['user_id', 'business_id', 'stars']], reader)
test_set = test_data.build_full_trainset()
test_set = test_set.build_testset()

In [12]:
results = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    predictions = algorithm.fit(train_set).test(test_set)
    rmse = {'test_rmse': [accuracy.rmse(predictions)]}

    tmp = pd.DataFrame.from_dict(rmse).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    results.append(tmp)
    
pd.DataFrame(results).set_index('Algorithm').sort_values('test_rmse') 

RMSE: 1.3375
RMSE: 1.3406
RMSE: 1.4356
RMSE: 1.4359
RMSE: 1.7926
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.3376
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.3842
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.4356
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.4357
Estimating biases using als...
RMSE: 1.3379
RMSE: 1.4354


,test_rmse
Algorithm,
SVD,1.337518
KNNBaseline,1.337643
BaselineOnly,1.337854
SVDpp,1.340634
KNNBasic,1.384197
CoClustering,1.435389
SlopeOne,1.435618
KNNWithMeans,1.435630
KNNWithZScore,1.435708
